# Trade Manager

# Buy agent

In [ ]:
# # from config.loader import load_trading_system_config
# config = load_trading_system_config("config/data_config.yaml")
# print(config)

In [ ]:
# from agents.buy_agent_trainer import BuyAgentTrainer
# from agents.sell_agent_trainer import SellAgentTrainer

# buy_trainer = BuyAgentTrainer(
#     ticker="AAPL",
#     window_size=30,
#     horizon=20,
#     transaction_cost=0.001,
#     lambda_dd=0.05,
#     lambda_vol=0.01,
#     hold_penalty_long=0.0,
#     device="cpu",
# )

# buy_trend_history = buy_trainer.train_trend_filtered(
#     n_episodes=200,
#     verbose=True,
# )


# Sell Agent

In [ ]:
# # 1) Build SellTrainer
# sell_trainer = SellAgentTrainer(
#     ticker="AAPL",
#     window_size=30,
#     horizon=20,
#     transaction_cost=0.001,
#     min_steps_before_sell=1,
#     lambda_dd=0.05,
#     lambda_vol=0.01,
#     hold_penalty_long=0.0,
#     device="cpu",
# )

# # 2) Train SellAgent on BuyAgent entries
# sell_history = sell_trainer.train_on_buy_entries(
#     buy_agent=buy_trainer.agent,
#     n_episodes=300,
#     verbose=True,
# )


In [ ]:
# from scripts.trade_manager import TradeManager
# # from agents.buy_agent_trainer import BuyAgentTrainer
# # from agents.sell_agent_trainer import SellAgentTrainer
# import numpy as np

# tm = TradeManager(
#     buy_trainer=buy_trainer,
#     sell_trainer=sell_trainer,
#     cfg_path="config/data_config.yaml",
# )

# print("Trend filter True count:", np.sum(tm.sma_short > tm.sma_long))

# # Inspect BuyAgent confidence over entire dataset
# import torch
# confs = []
# for i in range(len(tm.state_df)):
#     state = tm.state_df.iloc[i].values.astype(np.float32)
#     with torch.no_grad():
#         s = torch.from_numpy(state).unsqueeze(0).to(tm.buy_agent.device)
#         q = tm.buy_agent.q_net(s)[0].cpu().numpy()
#         p = np.exp(q - q.max()) / np.exp(q - q.max()).sum()
#         confs.append(p[1])

# print("Average BUY confidence:", np.mean(confs))
# print("Max BUY confidence:", np.max(confs))
# print("Min BUY confidence:", np.min(confs))

# results = tm.run_backtest(greedy=True)



# equity = results["equity_curve"]
# trades = results["trades"]

# print("Trades executed:", len(trades))
# print("First 5 trades:")
# for t in trades[:5]:
#     print(t)

# print("Final equity:", equity[-1])


In [ ]:
# equity = results["equity_curve"]
# trades = results["trades"]

# print("Trades executed:", len(trades))
# print("First 5 trades:")
# for t in trades[:5]:
#     print(t)

# print("Final equity:", equity[-1])


In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12,5))
# plt.plot(equity)
# plt.title("Equity Curve (After MP Training)")
# plt.xlabel("Time")
# plt.ylabel("Equity")
# plt.grid(True)
# plt.show()


# After refactoring

In [1]:
import numpy as np

from config.loader import load_config
from agents.buy_agent_trainer import BuyAgentTrainer
from agents.sell_agent_trainer import SellAgentTrainer
from scripts.trade_manager import TradeManager, TradeManagerParams


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
config = load_config("config/config.yaml")
print(config.trade_manager)


TradeManagerConfig(cooldown_steps=5, sell_horizon=20, confidence_temp=0.01, buy_min_confidence=0.5, use_trend_filter=True, ma_short=10, ma_long=30, min_hold_bars=2)


In [ ]:
# buy_trainer = BuyAgentTrainer(ticker="AAPL", config=config, device="cpu")
# buy_hist = buy_trainer.train(n_episodes=20, warmup_dynamic=True, verbose=True)

# buy_entry_indices = buy_trainer.collect_buy_entry_indices(
#     buy_min_confidence=0.48,   # tune here
#     use_trend_filter=False
# )

# print("Collected BUY entries:", len(buy_entry_indices))
# print("First 10:", buy_entry_indices[:10])

# sell_trainer = SellAgentTrainer(ticker="AAPL", config=config, device="cpu")
# sell_hist = sell_trainer.train(
#     # n_episodes=50,
#     n_episodes=300,
#     warmup_dynamic=True,
#     verbose=True,
#     buy_entry_indices=buy_entry_indices,
# )
# print("Sell final reward:", sell_hist[-1])


In [ ]:
# s = buy_trainer.env.reset()
# for _ in range(5):
#     s, r, d, info = buy_trainer.env.step(buy_trainer.env.action_space.sample())
# print("ok", r, d, info)

# Trade Manager

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scripts.trade_manager import TradeManager

# --------------------------------------------------
# 1. Retrieve trained agents
# --------------------------------------------------
buy_agent = buy_trainer.agent
sell_agent = sell_trainer.agent

# Safety checks
assert buy_agent is not None, "Buy agent not trained"
assert sell_agent is not None, "Sell agent not trained"

# --------------------------------------------------
# 2. Retrieve shared market data
#    (important: SAME data for both agents)
# --------------------------------------------------
state_df = buy_trainer.state_df
prices = buy_trainer.prices
# feature_cols = buy_trainer.get_feature_cols()

assert len(state_df) == len(prices)

print("state_df type:", type(state_df))
print("row type:", type(state_df.iloc[0]))
print("row shape:", state_df.iloc[0].values.shape)
print("agent state_dim:", buy_agent.state_dim)



# # --------------------------------------------------
# # 3. Instantiate TradeManager
# # --------------------------------------------------
# trade_manager = TradeManager(
#     buy_agent=buy_agent,
#     sell_agent=sell_agent,
#     state_df=state_df,
#     prices=prices,
#     config=config,
# )

# # --------------------------------------------------
# # 4. Run trading simulation
# # --------------------------------------------------
# results = trade_manager.run()

# # --------------------------------------------------
# # 5. Inspect results
# # --------------------------------------------------
# print("\n========== TRADE MANAGER RESULTS ==========")
# print(f"Final equity: {results['final_equity']:.4f}")
# print(f"Trades executed: {len(results['trades'])}")

# if len(results["trades"]) > 0:
#     print("First 5 trades:")
#     for t in results["trades"][:5]:
#         print(t)

# # --------------------------------------------------
# # 6. Plot equity curve
# # --------------------------------------------------
# plt.figure(figsize=(10, 4))
# plt.plot(results["equity_curve"])
# plt.title("TradeManager Equity Curve")
# plt.xlabel("Time step")
# plt.ylabel("Equity")
# plt.grid(True)
# plt.show()


In [ ]:
def compare_trade_managers(
    buy_agent, sell_agent, state_df, prices, config,
    force_close_at_end=False,   # set False to match old behavior exactly
    max_hold_bars=None          # set None to match old behavior exactly
):
    # OLD
    old_tm = TradeManagerOld(
        buy_agent=buy_agent,
        sell_agent=sell_agent,
        state_df=state_df,
        prices=prices,
        config=config,
    )
    old_res = old_tm.run()

    # NEW (disable new behaviors to match old)
    # params = TradeManagerParams(
    #     cooldown_steps=int(config.trade_manager.cooldown_steps),
    #     transaction_cost_round_trip=float(config.reward.transaction_cost),
    #     max_hold_bars=max_hold_bars,
    #     force_close_at_end=force_close_at_end,
    # )
    params = TradeManagerParams(
        cooldown_steps=int(config.trade_manager.cooldown_steps),
        transaction_cost_round_trip=float(config.reward.transaction_cost),
        max_hold_bars=None,
        force_close_at_end=False,
        buy_min_confidence=None,
        use_trend_filter=False,
    )

    new_tm = TradeManager(
        buy_agent=buy_agent,
        sell_agent=sell_agent,
        state_df=state_df,
        prices=prices,
        config=config,
        params=params,
    )
    new_res = new_tm.run()

    # --- comparisons ---
    print("FINAL EQUITY")
    print("old:", old_res["final_equity"])
    print("new:", new_res["final_equity"])
    print("abs diff:", abs(old_res["final_equity"] - new_res["final_equity"]))

    print("\nNUM TRADES")
    print("old:", len(old_res["trades"]))
    print("new:", len(new_res["trades"]))

    # Compare trade-by-trade (best effort)
    min_n = min(len(old_res["trades"]), len(new_res["trades"]))
    mismatches = 0
    for i in range(min_n):
        o = old_res["trades"][i]
        n = new_res["trades"][i]
        # old doesn't have gross_return/hold_bars/forced_exit; compare common keys
        keys = ["entry_idx", "exit_idx", "entry_price", "exit_price", "net_return"]
        for k in keys:
            if abs(float(o[k]) - float(n[k])) > 1e-12:
                mismatches += 1
                print(f"\nMismatch trade {i} field {k}: old={o[k]} new={n[k]}")
                break

    print("\nTRADE MISMATCH COUNT:", mismatches)
    return old_res, new_res


In [ ]:
# old_res, new_res = compare_trade_managers(
#     buy_agent, sell_agent, state_df, prices, config,
#     force_close_at_end=False,
#     max_hold_bars=int(config.trade_manager.sell_horizon),
# )


In [ ]:
params = TradeManagerParams(
        cooldown_steps=int(config.trade_manager.cooldown_steps),
        transaction_cost_round_trip=float(config.reward.transaction_cost),
        confidence_temp=float(config.trade_manager.confidence_temp),
        max_hold_bars=None,
        force_close_at_end=False,
        buy_min_confidence=None,
        use_trend_filter=False,
    )

new_tm = TradeManager(
    buy_agent=buy_agent,
    sell_agent=sell_agent,
    state_df=state_df,
    prices=prices,
    config=config,
    params=params,
)
new_res = new_tm.run()

In [ ]:
# # res = new_tm.run()
# trades = new_res["trades"]

# # 1) Equity curve length matches data
# assert len(new_res["equity_curve"]) == len(prices)

# # 2) Trades have increasing indices, and valid prices
# for tr in trades:
#     assert tr["entry_idx"] >= 0
#     assert tr["exit_idx"] >= tr["entry_idx"]
#     assert tr["entry_price"] > 0
#     assert tr["exit_price"] > 0

# # 3) Hold bars matches indices
# for tr in trades:
#     assert tr["hold_bars"] == tr["exit_idx"] - tr["entry_idx"]

# # 4) Net return matches gross - cost (your semantics)
# tc = float(config.reward.transaction_cost)
# for tr in trades:
#     approx = tr["gross_return"] - tc
#     assert abs(tr["net_return"] - approx) < 1e-10


In [ ]:
# class NeverSellAgent:
#     def __init__(self, state_dim): self.state_dim = state_dim
#     def select_action(self, state, greedy=True): return 0  # never sell

# # keep your buy_agent as-is, replace sell_agent
# sell_agent_never = NeverSellAgent(buy_agent.state_dim)

# params = TradeManagerParams(
#     cooldown_steps=int(config.trade_manager.cooldown_steps),
#     transaction_cost_round_trip=float(config.reward.transaction_cost),
#     max_hold_bars=None,
#     force_close_at_end=True,
# )

# tm = TradeManager(buy_agent, sell_agent_never, state_df, prices, config, params=params)
# res = tm.run()

# print("num trades:", len(res["trades"]))
# print("last trade forced_exit:", res["trades"][-1]["forced_exit"] if res["trades"] else None)
# print("last trade reason:", res["trades"][-1]["meta"].get("reason") if res["trades"] else None)


In [ ]:
# params = TradeManagerParams(
#     cooldown_steps=int(config.trade_manager.cooldown_steps),
#     transaction_cost_round_trip=float(config.reward.transaction_cost),
#     max_hold_bars=int(config.trade_manager.sell_horizon),  # 20
#     force_close_at_end=False,
# )

# tm = TradeManager(buy_agent, sell_agent_never, state_df, prices, config, params=params)
# res = tm.run()

# # count forced time exits
# time_exits = [t for t in res["trades"] if t["forced_exit"] and t["meta"].get("reason") == "time"]
# print("forced time exits:", len(time_exits))
# print("example hold_bars:", time_exits[0]["hold_bars"] if time_exits else None)


In [ ]:
# res = tm.run()
confs = [t["meta"].get("buy_confidence") for t in new_res["trades"] if t["meta"].get("buy_confidence") is not None]
print("trades:", len(new_res["trades"]))
print("min conf:", min(confs) if confs else None)
print("avg conf:", sum(confs)/len(confs) if confs else None)
print("p95 conf:", np.quantile(confs, 0.95) if confs else None)


# Last one

In [ ]:
buy_trainer = BuyAgentTrainer("AAPL", config, device="cpu")
buy_trainer.train(n_episodes=200)

# buy_entry_indices = buy_trainer.collect_buy_entry_indices()
# print("Num buy entries:", len(buy_entry_indices))
# print("First 10:", buy_entry_indices[:10])
from utils.entry_indices import collect_buy_signal_indices

In [ ]:
states = buy_trainer.state_df.values.astype(np.float32)

buy_count = 0
for i in range(len(states)):
    a = buy_trainer.agent.select_action(states[i], greedy=True)
    buy_count += int(a == 1)

print("Greedy BUY rate:", buy_count, "/", len(states), "=", buy_count/len(states))


In [ ]:
buy_entry_indices = buy_trainer.collect_buy_entry_indices(greedy=True, buy_min_confidence=None, use_trend_filter=False)
print("collect_buy_entry_indices (broad) =", len(buy_entry_indices), buy_entry_indices[:10])


In [ ]:
from utils.entry_indices import collect_buy_signal_indices

buy_entry_indices_train = collect_buy_signal_indices(
    buy_agent=buy_trainer.agent,
    state_df=buy_trainer.state_df.values.astype(np.float32),
    prices=np.asarray(buy_trainer.prices, dtype=np.float32),
    buy_min_confidence=None,
    use_trend_filter=False,
    confidence_temp=0.01,
    min_gap=5,                 # start with 5
)
print("TRAIN signal entries:", len(buy_entry_indices_train), buy_entry_indices_train[:10])

# states = buy_trainer.state_df.values.astype(np.float32)

# buy_count = 0
# for i in range(len(states)):
#     a = buy_trainer.agent.select_action(states[i], greedy=True)
#     buy_count += int(a == 1)

# print("Greedy BUY rate:", buy_count, "/", len(states), "=", buy_count/len(states))


In [ ]:
sell_trainer = SellAgentTrainer("AAPL", config, device="cpu")
sell_hist = sell_trainer.train(
    n_episodes=300,
    warmup_dynamic=True,
    verbose=True,
    buy_entry_indices=buy_entry_indices_train.tolist(),
)


In [ ]:
tm = TradeManager(
    buy_agent=buy_trainer.agent,
    sell_agent=sell_trainer.agent,
    state_df=states,     # whatever your TM expects now
    prices=prices,
    config=config,
)
res = tm.run()
print(res["final_equity"], len(res["trades"]))


In [ ]:
def debug_tm_entries(tm, max_print=20):
    n = len(tm.prices)
    enter_candidates = 0
    entered = 0

    reasons = {"action!=BUY":0, "conf_gate":0, "trend_gate":0, "cooldown":0}

    for t in range(n):
        # mimic the same entry checks used by tm
        if tm.in_position:
            continue
        if tm._cooldown > 0:
            reasons["cooldown"] += 1
            tm._cooldown -= 1
            continue

        state = tm._get_state(t)

        action, conf, _ = tm._buy_action_and_conf(state)
        if action != 1:
            reasons["action!=BUY"] += 1
            continue

        enter_candidates += 1

        if tm.params.buy_min_confidence is not None and conf is not None and conf < tm.params.buy_min_confidence:
            reasons["conf_gate"] += 1
            continue

        if tm.params.use_trend_filter and not tm._trend_ok(t):
            reasons["trend_gate"] += 1
            continue

        entered += 1
        if entered <= max_print:
            print("ENTER @", t, "conf", conf, "trend_ok", tm._trend_ok(t))

    print("\nCandidates (action==BUY):", enter_candidates)
    print("Entered after gates:", entered)
    print("Reasons:", reasons)


In [ ]:
tm.reset()
debug_tm_entries(tm)
res = tm.run()
print("Trades:", len(res["trades"]), "Equity:", res["final_equity"])



In [ ]:
from scripts.metrics import summarize_backtest
print(summarize_backtest(res))


In [ ]:
print("tm_cfg:", config.trade_manager)
print("min_conf:", config.trade_manager.buy_min_confidence)
print("trend:", config.trade_manager.use_trend_filter)
print("cooldown:", config.trade_manager.cooldown_steps)
print("min_hold:", getattr(config.trade_manager, "min_hold_bars", None))


In [ ]:
buy_entry_indices_train = generate_buy_entry_indices(
    buy_agent=buy_trainer.agent,
    state_df=buy_trainer.state_df.values.astype(np.float32),
    prices=np.asarray(buy_trainer.prices, dtype=np.float32),
    buy_min_confidence=None,     # broad
    use_trend_filter=False,      # broad
    cooldown_steps=5,
    confidence_temp=0.01,
)
print("TRAIN entries:", len(buy_entry_indices_train),
      "min/max:", buy_entry_indices_train.min(), buy_entry_indices_train.max(),
      "first10:", buy_entry_indices_train[:10])


In [ ]:
buy_entry_indices_eval = generate_buy_entry_indices(
    buy_agent=buy_trainer.agent,
    state_df=buy_trainer.state_df.values.astype(np.float32),
    prices=np.asarray(buy_trainer.prices, dtype=np.float32),
    buy_min_confidence=0.5,
    use_trend_filter=True,
    cooldown_steps=5,
    confidence_temp=0.01,
)
print("EVAL entries:", len(buy_entry_indices_eval),
      "min/max:", buy_entry_indices_eval.min(), buy_entry_indices_eval.max(),
      "first10:", buy_entry_indices_eval[:10])


In [ ]:
from utils.entry_indices import collect_buy_signal_indices

prices = np.asarray(buy_trainer.prices, dtype=np.float32)
states = buy_trainer.state_df.values.astype(np.float32)

buy_entry_indices_train = collect_buy_signal_indices(
    buy_agent=buy_trainer.agent,
    state_df=states,
    prices=prices,
    buy_min_confidence=None,     # BROAD
    use_trend_filter=False,      # BROAD
    confidence_temp=0.01,
    min_gap=1,                   # keep everything
)
print("TRAIN signal entries:", len(buy_entry_indices_train),
      "min/max:", buy_entry_indices_train.min(), buy_entry_indices_train.max())


In [ ]:
# 1) Train BUY (with TM checkpointing enabled)
buy_trainer = BuyAgentTrainer("AAPL", config, device="cpu")
buy_trainer.train(n_episodes=200)

# 2) Generate SELL training entries (signals, broad)
from utils.entry_indices import collect_buy_signal_indices
states_np = buy_trainer.state_df.values.astype(np.float32)
prices_np = np.asarray(buy_trainer.prices, dtype=np.float32)

buy_entry_indices_train = collect_buy_signal_indices(
    buy_agent=buy_trainer.agent,
    state_df=states_np,
    prices=prices_np,
    buy_min_confidence=None,
    use_trend_filter=False,
    confidence_temp=0.01,
    min_gap=5,
)
print("TRAIN signal entries:", len(buy_entry_indices_train), buy_entry_indices_train[:10])

# 3) Train SELL
sell_trainer = SellAgentTrainer("AAPL", config, device="cpu")
sell_trainer.train(
    n_episodes=300,
    warmup_dynamic=True,
    verbose=True,
    buy_entry_indices=buy_entry_indices_train.tolist(),
)

# 4) Evaluate end-to-end
tm = TradeManager(
    buy_agent=buy_trainer.agent,
    sell_agent=sell_trainer.agent,
    state_df=buy_trainer.state_df,
    prices=buy_trainer.prices,
    config=config,
)
res = tm.run()
print("Final equity:", res["final_equity"], "Trades:", len(res["trades"]))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[BuyTrainer] Raw dataset: (1224, 10)
[BuyTrainer] After dropna: (1224, 10)
[BuyTrainer] Rolling state_df shape: (1194, 270)
[BuyTrainer] state_dim=270, actions=2
[BuyTrainer] Warmup set to: 238
[Buy Ep 1/20] Reward=1.1043 | Eps=0.773 | Steps=1193 | Buffer=1193 | Avg10=1.1043
[BuyTrainer] TM-checkpoint score=2.378205 metrics={'final_equity': 2.522308238422977, 'n_trades': 39, 'max_drawdown': -0.28820564780473}
[BuyTrainer] ✅ New BEST checkpoint at ep=10: {'final_equity': 2.522308238422977, 'n_trades': 39, 'max_drawdown': -0.28820564780473}
[Buy Ep 10/20] Reward=1.5271 | Eps=0.050 | Steps=1193 | Buffer=11930 | Avg10=1.2559
[BuyTrainer] TM-checkpoint score=1.126193 metrics={'final_equity': 1.20898309438292, 'n_trades': 11, 'max_drawdown': -0.16557969157423902}
[Buy Ep 20/20] Reward=1.1544 | Eps=0.050 | Steps=1193 | Buffer=23860 | Avg10=1.0417
[BuyTrainer] Restored BEST checkpoint: {'final_equity': 2.522308238422977, 'n_trades': 39, 'max_drawdown': -0.28820564780473} (score=2.378205)


AttributeError: 'DDQNAgent' object has no attribute 'q_values'

In [ ]:
from collections import Counter

reasons = Counter(t.get("meta", {}).get("reason") for t in res["trades"])
forced = sum(bool(t["forced_exit"]) for t in res["trades"])

print("exit reasons:", reasons)
print("forced %:", forced / max(1, len(res["trades"])))
